In [1]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import epfl_campus
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp
from mvpose.data import umpm

root = join(Settings['data_root'], 'pak')
user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']
tmp = Settings['tmp']
tmp = '/tmp'

params = get_settings()
loader = Loader(with_gpu=True)

import mvpose.data.skeleton_augmentation as ska
model_path = '../data/model_poseprediction.h5'
gen = ska.LimbGenerator(model_path, params.scale_to_mm)

X, Y, Calib = umpm.get(root, 'p2_free_1', user, pwd)

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


UMPM


In [2]:
def generate_pcp_score(frame):
    alpha = 0.3
    Im = np.array([X[0][frame], X[1][frame], X[2][frame], X[3][frame]])
    heatmaps, pafs = loader.load_confidence_map_and_paf('umpm', 
                                                            Im, 
                                                            frame, 
                                                            dir=tmp)
    
    detections = pose.estimate(
        Calib, heatmaps, pafs, settings=params, debug=False)
    
    detections = gen.apply(detections)
    
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
    a = Y[frame,0:14,0:3]
    b = Y[frame,14:,0:3]
    Y_ = kth.transform3d_from_umpm([a, b])
    
    for gtid, gt in enumerate(Y_):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs, detections


Detections = []
PER_GTID = {}
n = 2437
#n = 20
for frame in range(0, n, 5):
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs, detections =\
        generate_pcp_score(frame)
    Detections.append(detections)
    _end = time()
    print('frame ' + str(frame) + ', elapsed', _end - _start)
    
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)

frame 0, elapsed 8.208523750305176
frame 5, elapsed 4.22887659072876
frame 10, elapsed 4.376387357711792
frame 15, elapsed 4.380504131317139
frame 20, elapsed 4.387178659439087
frame 25, elapsed 4.1659486293792725
frame 30, elapsed 4.189700126647949
frame 35, elapsed 4.161163568496704
frame 40, elapsed 4.292321681976318
frame 45, elapsed 4.339201211929321
frame 50, elapsed 4.208050727844238
frame 55, elapsed 4.319344758987427
frame 60, elapsed 4.237339735031128
frame 65, elapsed 4.14666748046875
frame 70, elapsed 4.137843370437622
frame 75, elapsed 4.536255836486816
frame 80, elapsed 5.214984178543091
frame 85, elapsed 4.814350605010986
frame 90, elapsed 5.005974531173706
frame 95, elapsed 4.466122627258301
frame 100, elapsed 4.518089771270752
frame 105, elapsed 4.172976970672607
frame 110, elapsed 4.264025688171387
frame 115, elapsed 4.318525791168213
frame 120, elapsed 4.507330656051636
frame 125, elapsed 4.489558935165405
frame 130, elapsed 4.451192617416382
frame 135, elapsed 4.201

frame 1110, elapsed 4.495130777359009
frame 1115, elapsed 4.361050367355347
frame 1120, elapsed 4.441422939300537
frame 1125, elapsed 4.355167388916016
frame 1130, elapsed 4.285948991775513
frame 1135, elapsed 4.495675086975098
frame 1140, elapsed 4.3639795780181885
frame 1145, elapsed 4.599035739898682
frame 1150, elapsed 4.93129563331604
frame 1155, elapsed 4.684307098388672
frame 1160, elapsed 4.514784574508667
frame 1165, elapsed 4.592463731765747
frame 1170, elapsed 7.9168479442596436
frame 1175, elapsed 6.730603218078613
frame 1180, elapsed 8.22049593925476
frame 1185, elapsed 4.7116334438323975
frame 1190, elapsed 4.4286463260650635
frame 1195, elapsed 4.380728721618652
frame 1200, elapsed 4.233412265777588
frame 1205, elapsed 4.226707696914673
frame 1210, elapsed 4.437936782836914
frame 1215, elapsed 4.378776788711548
frame 1220, elapsed 4.350354194641113
frame 1225, elapsed 4.241676568984985
frame 1230, elapsed 4.230813026428223
frame 1235, elapsed 4.218045234680176
frame 1240

frame 2190, elapsed 4.191653251647949
frame 2195, elapsed 4.208483934402466
frame 2200, elapsed 4.211578130722046
frame 2205, elapsed 4.310612678527832
frame 2210, elapsed 4.214258432388306
frame 2215, elapsed 4.2032575607299805
frame 2220, elapsed 4.227991342544556
frame 2225, elapsed 4.201912879943848
frame 2230, elapsed 4.31929349899292
frame 2235, elapsed 4.295961141586304
frame 2240, elapsed 4.585596084594727
frame 2245, elapsed 5.027601003646851
frame 2250, elapsed 4.702749967575073
frame 2255, elapsed 4.628800868988037
frame 2260, elapsed 4.4857072830200195
frame 2265, elapsed 4.329989194869995
frame 2270, elapsed 4.210740566253662
frame 2275, elapsed 4.135658502578735
frame 2280, elapsed 4.401391983032227
frame 2285, elapsed 4.322908639907837
frame 2290, elapsed 4.309970378875732
frame 2295, elapsed 4.23045802116394
frame 2300, elapsed 4.239802122116089
frame 2305, elapsed 4.154715061187744
frame 2310, elapsed 4.175324201583862
frame 2315, elapsed 4.31218409538269
frame 2320, e

In [3]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

actor  0
	uarms: 0.9426229508196722
	larms: 0.9098360655737705
	ulegs: 0.9887295081967213
	llegs: 0.9743852459016393
	avg:   0.9538934426229508
actor  1
	uarms: 0.9323770491803278
	larms: 0.8944672131147541
	ulegs: 0.9805327868852459
	llegs: 0.9682377049180327
	avg:   0.9439036885245902

avg*:   0.9488985655737705


In [4]:
# ALPHA = 0.3
# params = get_settings()

# NO PC
# actor  0
# 	uarms: 0.9426229508196722
# 	larms: 0.9098360655737705
# 	ulegs: 0.9887295081967213
# 	llegs: 0.9743852459016393
# 	avg:   0.9538934426229508
# actor  1
# 	uarms: 0.9323770491803278
# 	larms: 0.8944672131147541
# 	ulegs: 0.9805327868852459
# 	llegs: 0.9682377049180327
# 	avg:   0.9439036885245902

# avg*:   0.9488985655737705

## Like in the Master thesis
### alpha=0.5
* actor  0
	* uarms: 0.9764344262295082
	* larms: 0.9641393442622951
	* ulegs: 0.9897540983606558
	* llegs: 0.985655737704918
	* avg:   0.9789959016393442
* actor  1
	* uarms: 0.9682377049180327
	* larms: 0.9334016393442623
	* ulegs: 0.9877049180327869
	* llegs: 0.9805327868852459
	* avg:   0.967469262295082
* avg*:   0.9732325819672132

### alpha=0.2
* actor  0
	* uarms: 0.7510245901639344
	* larms: 0.6875
	* ulegs: 0.9743852459016393
	* llegs: 0.867827868852459
	* avg:   0.8201844262295082
* actor  1
	* uarms: 0.6997950819672131
	* larms: 0.7008196721311475
	* ulegs: 0.9323770491803278
	* llegs: 0.9293032786885246
	* avg:   0.8155737704918034
* avg*:   0.8178790983606558

## Like in Master thesis without Unaries
### alpha=0.5
* actor  0
	* uarms: 0.9774590163934426
	* larms: 0.9651639344262295
	* ulegs: 0.9897540983606558
	* llegs: 0.9866803278688525
	* avg:   0.979764344262295
* actor  1
	* uarms: 0.9692622950819673
	* larms: 0.9323770491803278
	* ulegs: 0.9877049180327869
	* llegs: 0.9805327868852459
	* avg:   0.967469262295082
* avg*:   0.9736168032786885

### alpha=0.2
* actor  0
	* uarms: 0.7510245901639344
	* larms: 0.6875
	* ulegs: 0.9733606557377049
	* llegs: 0.8668032786885246
	* avg:   0.819672131147541
* actor  1
	* uarms: 0.6987704918032787
	* larms: 0.6997950819672131
	* ulegs: 0.9313524590163934
	* llegs: 0.9282786885245902
	* avg:   0.8145491803278688
* avg*:   0.8171106557377049